In [9]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# import tensorflow as tf
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
# print('TensorFlow version =',tf.__version__)

In [18]:
from utility import UNet_layer
from UNet_HCRF_model import unet_global

NotFoundError: dlopen(/Users/maxiaohang/miniforge3/lib/python3.9/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN10tensorflow16TensorShapeProtoC1ERKS0_
  Referenced from: <10B7FC95-0B10-3E4E-84D0-79A2D52E4D78> /Users/maxiaohang/miniforge3/lib/python3.9/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <70E47AF7-0018-34CB-8676-A89EF044ED50> /Users/maxiaohang/miniforge3/lib/python3.9/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so

In [ ]:
! pip3 install importlib

In [ ]:
! pip install -U git+https://github.com/albumentations-team/albumentations

In [4]:
# READ ALL SPECTROGRAMS
# PATH = './dataset/train_spectrograms/'
# files = os.listdir(PATH)
# print(f'There are {len(files)} spectrogram parquets')


FileNotFoundError: [Errno 2] No such file or directory: './dataset/train_spectrograms/'

In [5]:
# READ ALL SPECTROGRAMS
# PATH = './dataset/EEG_Spectrograms/'
# files = os.listdir(PATH)
# print(f'There are {len(files)} EEG spectrogram parquets')


FileNotFoundError: [Errno 2] No such file or directory: './dataset/EEG_Spectrograms/'

In [10]:
# train = pd.read_csv('./dataset/train.csv')

################
df = pd.read_csv('./dataset/train.csv')
TARGETS = df.columns[-6:]

train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spectrogram_id','spec_min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['spec_max'] = tmp

tmp = df.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds']].agg(
    {'eeg_label_offset_seconds':'min' }
)
train['eeg_min'] = tmp

tmp = df.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds']].agg(
    {'eeg_label_offset_seconds':'max' }
)
train['eeg_max'] = tmp

tmp = df.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds']].agg(
    {'eeg_label_offset_seconds':'mean' }
)
train['eeg_mean'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
train['expert_numner'] = tmp.sum(axis=1)
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()
####################

TARGETS = train.columns[-7:-1]
print('Train shape:', train.shape )
print('Targets', list(TARGETS))
print('Unique EEG id:', train.eeg_id.unique().shape)
print('Spectrogram id:', train.spectrogram_id.unique().shape)
print('Unique EEG_Spectrogram id:', train.groupby(['eeg_id', 'spectrogram_id']).agg('first').shape[0])
# tmp_index = train.eeg_label_offset_seconds > train.spectrogram_label_offset_seconds
# print('Eeg offset time longer: ', train[tmp_index].shape[0])
# tmp_index = train.eeg_label_offset_seconds < train.spectrogram_label_offset_seconds
# print('Spectrogram offset time longer: ', train[tmp_index].shape[0])
tmp = train.groupby('eeg_id').apply(lambda x: len(np.unique(x['spectrogram_id'].values)))
print(f'Non-unique spectrogram id upper bound:{tmp.max()}')
tmp = train.groupby('spectrogram_id').apply(lambda x: len(np.unique(x['eeg_id'].values)))
print(f'Non-unique eeg id upper bound:{tmp.max()}')
# pd.DataFrame.groupby()
train.head()


Train non-overlapp eeg_id shape: (17089, 16)
Train shape: (17089, 16)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Unique EEG id: (17089,)
Spectrogram id: (11138,)
Unique EEG_Spectrogram id: 17089
Non-unique spectrogram id upper bound:1
Non-unique eeg id upper bound:107


,eeg_id,spectrogram_id,spec_min,spec_max,eeg_min,eeg_max,eeg_mean,patient_id,expert_numner,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,0.0,16.0,8.500000,20654,48,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,0.0,38.0,17.454545,20230,154,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,0.0,24.0,12.000000,5955,2,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,0.0,0.0,0.000000,38549,1,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,0.0,0.0,0.000000,40955,2,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [11]:
train[train.eeg_id == 642382]

,eeg_id,spectrogram_id,spec_min,spec_max,eeg_min,eeg_max,eeg_mean,patient_id,expert_numner,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
2,642382,14960202,1008.0,1032.0,0.0,24.0,12.0,5955,2,0.0,0.0,0.0,0.0,0.0,1.0,Other


### Loading EEG_SPEC and SPEC data

In [17]:
eeg = np.load('./dataset/non_overlap_eeg_specs_non_denoise_sample.npy', allow_pickle=True).item()
spectrograms = np.load('./dataset/spec_sample.npy',allow_pickle=True).item()

### Dataloader for training

In [16]:
import albumentations as albu
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
TARS2 = {x:y for y,x in TARS.items()}

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32, shuffle=False, augment=False, mode='train',
                 specs = spectrograms, eeg_specs = all_eegs):

        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mode = mode
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = int( np.ceil( len(self.data) / self.batch_size ) )
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        if self.augment: X = self.__augment_batch(X)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange( len(self.data) )
        if self.shuffle: np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'

        X = np.zeros((len(indexes),128,256,8),dtype='float32')
        y = np.zeros((len(indexes),6),dtype='float32')
        img = np.ones((128,256),dtype='float32')

        for j,i in enumerate(indexes):
            row = self.data.iloc[i]
            if self.mode=='test':
                r = 0
            else:
                r = int( (row['min'] + row['max'])//4 )

            for k in range(4):
                # EXTRACT 300 ROWS OF SPECTROGRAM
                img = self.specs[row.spec_id][r:r+300,k*100:(k+1)*100].T

                # LOG TRANSFORM SPECTROGRAM
                img = np.clip(img,np.exp(-4),np.exp(8))
                img = np.log(img)

                # STANDARDIZE PER IMAGE
                ep = 1e-6
                m = np.nanmean(img.flatten())
                s = np.nanstd(img.flatten())
                img = (img-m)/(s+ep)
                img = np.nan_to_num(img, nan=0.0)

                # CROP TO 256 TIME STEPS
                X[j,14:-14,:,k] = img[:,22:-22] / 2.0

            # EEG SPECTROGRAMS
            img = self.eeg_specs[row.eeg_id]
            X[j,:,:,4:] = img

            if self.mode!='test':
                y[j,] = row[TARGETS]

        return X,y

    def __random_transform(self, img):
        composition = albu.Compose([
            albu.HorizontalFlip(p=0.5),
            #albu.CoarseDropout(max_holes=8,max_height=32,max_width=32,fill_value=0,p=0.5),
        ])
        return composition(image=img)['image']

    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i, ] = self.__random_transform(img_batch[i, ])
        return img_batch

/Users/maxiaohang/miniforge3/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


NameError: name 'tf' is not defined

### You may need to integrate the build_model() below with our unet_global() model 

In [ ]:
USE_KAGGLE_SPECTROGRAMS = True
USE_EEG_SPECTROGRAMS = True
def build_model():

    inp = tf.keras.Input(shape=(128,256,8))
    base_model = None 
    
    # RESHAPE INPUT 128x256x8 => 512x512x3 MONOTONE IMAGE
    # KAGGLE SPECTROGRAMS
    x1 = [inp[:,:,:,i:i+1] for i in range(4)]
    x1 = tf.keras.layers.Concatenate(axis=1)(x1)
    # EEG SPECTROGRAMS
    x2 = [inp[:,:,:,i+4:i+5] for i in range(4)]
    x2 = tf.keras.layers.Concatenate(axis=1)(x2)
    # MAKE 512X512X3
    if USE_KAGGLE_SPECTROGRAMS & USE_EEG_SPECTROGRAMS:
        x = tf.keras.layers.Concatenate(axis=2)([x1,x2])
    elif USE_EEG_SPECTROGRAMS: x = x2
    else: x = x1
    x = tf.keras.layers.Concatenate(axis=3)([x,x,x])

    # OUTPUT
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)

    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()

    model.compile(loss=loss, optimizer = opt)

    return model